In [8]:
###########Script pour créer la base de données, exécuter les requêtes SQL dans le conteneur Docker, extraire les résultats et les sauvegarder dans des fichiers CSV.###########

In [9]:
import requests
import os
import gzip
from tqdm.notebook import trange
import subprocess


In [10]:
# Demarrer le conteneur Docker pour MySQL
print("🚀 Starting Docker container...")
COMPOSE_FILE="../docker/docker-compose.yml"
subprocess.run(["docker", "compose", "-f", COMPOSE_FILE, "up", "-d", "--build"])


🚀 Starting Docker container...


 Image docker-mysql Building 


#1 [internal] load local bake definitions
#1 reading from stdin 558B done
#1 DONE 0.0s

#2 [internal] load build definition from Dockerfile.mysql
#2 transferring dockerfile: 285B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/mysql:8.0
#3 ...

#4 [auth] library/mysql:pull token for registry-1.docker.io
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/mysql:8.0
#3 DONE 3.4s

#5 [internal] load .dockerignore
#5 transferring context: 2B done
#5 DONE 0.0s

#6 [1/1] FROM docker.io/library/mysql:8.0@sha256:99d774bf02a48a1bb1c599920d2571946d31e5940b854b02737d5e95c184358f
#6 resolve docker.io/library/mysql:8.0@sha256:99d774bf02a48a1bb1c599920d2571946d31e5940b854b02737d5e95c184358f done
#6 CACHED

#7 exporting to image
#7 exporting layers done
#7 exporting manifest sha256:cc0547b9a74fe963a4486a74cf94975e891023bb7340e940939d9030dc7715c7 done
#7 exporting config sha256:c5bfc23de452870463110aea77e9e84417f5e2bb10e662033d28b4d55c2c6b54 done
#7 exporting attestati

 Image docker-mysql Built 
 Network docker_default Creating 
 Network docker_default Created 
 Container moviesdb_mysql Creating 
 Container moviesdb_mysql Created 
 Container moviesdb_mysql Starting 
 Container moviesdb_mysql Started 


CompletedProcess(args=['docker', 'compose', '-f', '../docker/docker-compose.yml', 'up', '-d', '--build'], returncode=0)

In [20]:
# subprocess.run : exécute une commande externe et renvoie un objet CompletedProcess.
# Arguments courants (explication) :
# - args (liste ou str) : commande et ses arguments. Préférer une liste pour éviter les injections.
# - capture_output=True : capture stdout et stderr.
# - text=True : renvoie stdout/stderr en tant que str au lieu de bytes.
# - check=True : si True, lève CalledProcessError quand le code de retour est non nul ; si False, il faut vérifier manuellement result.returncode.
# - shell=True : exécute la commande via le shell.
# Retour : CompletedProcess avec les attributs .args, .returncode, .stdout, .stderr.


#Modifier le nom du fichier pour chaque requête SQL
file_names = {"marvel_movies.sql","marvel_characters.sql","marvel_directors.sql","marvel_actors.sql"}
#Copier le fichier de requête dans le conteneur

for file in file_names:
    print(f"📁 Copied {file} into docker container")
    result = subprocess.run(["docker", "cp", f"../mysql/{file}", f"moviesdb_mysql:/tmp/{file}"], capture_output=True, text=True, check=False)
    print(result.stderr)

📁 Copied marvel_actors.sql into docker container

📁 Copied marvel_movies.sql into docker container

📁 Copied marvel_characters.sql into docker container

📁 Copied marvel_directors.sql into docker container



In [ ]:
# Executer la requête et exporter le CSV
csvs = [file.replace('.sql', '.csv') for file in file_names]
DB = "IMDb"  # database name inside the MySQL container
CONTAINER = "moviesdb_mysql"
# SQL_IN_CONTAINER = f"/tmp/{file_name}"  # path inside the container where the SQL file is located
# HOST_OUT_PATH = f"../data/tests/{output_name}"
# CONTAINER_OUT = f"/var/lib/mysql-files/{output_name}"  # full path inside the container for the output CSV

for csv in csvs:
    #supprimer le fichier csv s'il existe déjà
    result = subprocess.run([
        "docker", "exec", "-i", CONTAINER,
        "rm", "-f", f"/var/lib/mysql-files/{csv}"
    ], check=True)

#executer les requetes sql 
for file in file_names:
    try:
        result = subprocess.run([
            "docker", "exec", "-i",
            CONTAINER,
            "bash", "-c",
            f"mysql -u root -proot {DB} < /tmp/{file}"
        ], capture_output=True, text=True, check=False)
    except: 
        print("Something went wrong: ",result.returncode, result.stderr)



In [22]:
# #Copier le fichier dans le dossier local
# result = subprocess.run([
#     'docker', 'cp',
#     f'{CONTAINER}:{CONTAINER_OUT}',
#     HOST_OUT_PATH
# ], capture_output=True, text=True, check=False)

# if result.returncode != 0:
#     print("❌ Docker cp failed!")
#     print("STDERR:", result.stderr)
# else:
#     print("✅ File copied successfully")

In [23]:
#Lire le CSV avec pandas
import pandas as pd
import json
file_names = {"marvel_movies.sql","marvel_characters.sql","marvel_directors.sql","marvel_actors.sql"}

csvs = [file.replace('.sql', '.csv') for file in file_names]

def parse_character_list(x):
    """Parse JSON array into Python list, return None if invalid"""
    
    if pd.isna(x) or x == '\\N':
        return None
    x = x.strip('"')
    x = x.replace('""', '"')
    print(x)
    try:
        parsed = json.loads(x)
        print(parsed[0])
        return parsed[0] if isinstance(parsed, list) and parsed else None
    except json.JSONDecodeError:
        return None

for csv in csvs:
    path = f"../data/tests/{csv}"

    # if "movies" in csv:
    #     columns_movies = ["title_id","primary_title","genres","start_year"]
    #     marvel_df = pd.read_csv(path, sep=",",
    #     header=0,
    #     encoding="utf-8",
    #     engine="python", names = columns_movies)

    # el
    if "characters" in csv: 
        columns_characters = ["character_name"]
        marvel_df = pd.read_csv(path, encoding='latin-1', header=0,   names= columns_characters)
        marvel_df["character_name"] = marvel_df["character_name"].str.strip('"').str.replace('""', '"', regex=False).apply(lambda x: json.loads(x)[0] if pd.notna(x) else None)
        marvel_df = marvel_df.explode("character_name").reset_index(drop=True)

    elif "directors" in csv:
        columns_directors = ["title_id", "person_id", "person_name", "job", "category"]
        marvel_df = pd.read_csv(path, sep=",",
        header=0,
        encoding="utf-8",
        engine="python", names= columns_directors)

    elif "actors" in csv:
        columns_actors = ["person_id", "person_name"]
        marvel_df = pd.read_csv(path, sep=",",
        header=0,
        encoding="utf-8",
        engine="python", names= columns_actors)
    marvel_df.to_csv(path, index=False)

print(marvel_df.head())
print(marvel_df.shape)
print("Returned rows:", len(marvel_df))




     title_id  person_id        person_name       job  category
0   tt5463162  nm0005351      Ryan Reynolds  producer  producer
1   tt1431045  nm0005351      Ryan Reynolds  producer  producer
2  tt22084616  nm0032696           Avi Arad  producer  producer
3   tt0371746  nm0032696           Avi Arad  producer  producer
4   tt5095030  nm0112780  Stephen Broussard  producer  producer
(107, 5)
Returned rows: 107


In [24]:
# #Arreter le conteneur Docker

print("🚀 Stopping Docker container")
COMPOSE_FILE="../docker/docker-compose.yml"
CONTAINER_NAME="moviesdb_mysql"

subprocess.run(["docker", "stop", CONTAINER_NAME])
subprocess.run(["docker", "compose", "-f", COMPOSE_FILE, "down"])

🚀 Stopping Docker container
moviesdb_mysql


 Container moviesdb_mysql Stopping 
 Container moviesdb_mysql Stopped 
 Container moviesdb_mysql Removing 
 Container moviesdb_mysql Removed 
 Network docker_default Removing 
 Network docker_default Removed 


CompletedProcess(args=['docker', 'compose', '-f', '../docker/docker-compose.yml', 'down'], returncode=0)